In [ ]:
# If we don't have the data, download it
!wget https://dmlab.cs.gsu.edu/solar/data/data-comp-2020/train_partition1_data.json
!mkdir data
!mv train_partition1_data.json data/

In [1]:
import pandas as pd
import numpy as np
import torch
import json
from normalizer import counter, subSample
from torch.utils.data import DataLoader
from normalizer import subSample, getDataFromJSON, trainer

device = 'cuda' if torch.cuda.is_available() else 'cpu' # Change this to cuda for GPU enabled computers
device

/home/robjohnson/.local/lib/python3.8/site-packages/torch/cuda/__init__.py:52: UserWarning: CUDA initialization: Found no NVIDIA driver on your system. Please check that you have an NVIDIA GPU and installed a driver from http://www.nvidia.com/Download/index.aspx (Triggered internally at  /pytorch/c10/cuda/CUDAFunctions.cpp:100.)
  return torch._C._cuda_getDeviceCount() > 0


'cpu'

In [2]:
train1, labels1 = subSample("data/train_partition1_data.json", device=device)

Now reading data/train_partition1_data.json
Now loading event 1/785
Now loading event 101/785
Now loading event 201/785
Now loading event 301/785
Now loading event 401/785
Now loading event 501/785
Now loading event 601/785
Now loading event 701/785
785 lines loaded.


In [4]:
# Define the network. Make sure to end with nn.Softmax activation
import torch.nn as nn
# from skorch import NeuralNet

class logRegWithHidden(nn.Module):
    def __init__(self, hidden_size1, hidden_size2, num_classes=5, drop1=.5, input_size=1980):
        super().__init__() 
        self.layer1 = nn.Linear(input_size, hidden_size1)
        self.layer2 = nn.Linear(hidden_size1, hidden_size2)
        self.layerout = nn.Linear(hidden_size2, num_classes)
        #Define a RELU Activation unit
        self.relu = nn.ReLU()  
        self.smax = nn.Softmax(dim=1)
        self.drop = nn.Dropout(p=drop1)

    def forward(self, x):
        #Forward Propagate through the layers as defined above
        y = self.drop(x.reshape(-1, 1980))
        y = self.drop(self.relu(self.layer1(y)))
        y = self.relu(self.layer2(y))
        y = self.layerout(y)
        y = self.smax(y)
        return y



In [47]:
# def train(modelModule, inputs, labels, weight, valSets, valLabels, valweight, *modelArgs, lr=0.0001, epochs=50,
#           pmodel = False, pstateDict = False, pindvLoss = False, pmodelDict = False, pvalidateOut = False,
#           pbl = False, checkClone = False, pbalApp = False, **modelKwargs):
    
#     '''
#     function call: 
#     train(modelModule, inputs, labels, weight, valSets, valLabels, valweight, *modelArgs, lr=0.0001, 
#           pmodel = False, pstateDict = False, pindvLoss = False, pmodelDict = False, pvalidateOut = False,
#           pbl = False, checkClone = False **modelKwargs):
    
#     modelModule is the nn.module form of the network you want trained
#     inputs and labels are the Xs and Ys respectively of the training data
#     weights are the weights to use for the cross entropy loss function - see more in counter documentation
#     valinputs, valLabels, and valweight are the same fields but for our validation set
#     lr is the learning rate for the optimizer
#     Epochs is the amount of times a network is trained on the data
#     pmodel prints the model if True, defaults to False
#     pstateDict prints the state dict for the optimizer if True, defaults to False
#     pindvLoss prints the individual loss for each minibatch if True, defaults to False
#     pmodelDict prints the state dictionary for the model if True, defaults to False
#     pvalidateOut prints all errors and accuracies of the validation set if True, defaults to False
#     pbl prints the batch loss at declaration time if True, defaults to False
#     checkClone prints the maximum and the minimum difference between the initial model and the current iteration 
#         of the model if True, defaults to False
#     pblApp checks the validation appending
#     modelArgs and modelKwargs are the arguments for the model and the keyword arguments of the model, repsectivley
#     '''

#     # Define the model
#     model = modelModule(*modelArgs, **modelKwargs)
#     # if we are going to check this against itself to make sure it is learning, we need to be able to come back
#     if checkClone: 
#         PATH = './.cloneModel.pth'
#         torch.save(model.state_dict(), PATH)
#     if pmodel: print(model)
    
#     # Define loss functions
#     if weight is not None: weight = torch.Tensor(weight)
#     lfc = nn.CrossEntropyLoss(weight=weight)
#     if valweight is not None: valweight = torch.Tensor(valweight)
#     valLoss = nn.CrossEntropyLoss(weight=valweight)

    

    
#     # Hyperparameters
#     batch = 64
    
#     # Start a dataloader object
#     data = list(zip(inputs,labels))
#     val = list(zip(valSets,valLabels))
#     loader = DataLoader(data, batch_size = batch, num_workers=4)
#     valLoader = DataLoader(val, batch_size = 1, num_workers=4)
#     opt = torch.optim.SGD(model.parameters(), lr=lr, momentum=0.25)
#     if pstateDict: print(opt.state_dict())

#     for epoch in range(epochs):
#         model.train()
#         batch_loss = []
#         if pbl: print(batch_loss)
#         for (xtrain,ytrain) in loader:
#             opt.zero_grad()
#             if pstateDict: print('Opt dict post zero grad:', '================',opt.state_dict(), sep='\n')
#             output = model(xtrain)
#             loss = lfc(output,ytrain)
#             if pindvLoss: print(loss)
#             loss.backward()
#             if pstateDict: print('Opt dict post backward:', '================',opt.state_dict(), sep='\n')
#             if pmodelDict: print(model.state_dict())
#             opt.step()
#             if pstateDict: print('Opt dict post step:', '================',opt.state_dict(), sep='\n')
#             if pstateDict or pmodelDict: print('\n\n\n\n\n')
#             batch_loss.append(loss.item())
#         print(f'The training loss for epoch {epoch+1}/{epochs} was {np.mean(batch_loss)}')
#         if pbl: print(batch_loss)
        
#         model.eval()
#         balanced = [[],[],[],[],[]]
#         batchLoss = []
#         unbalanced = []
        
#         for (xval,yval) in valLoader:
#             output = model(xval)
#             loss = valLoss(output,yval)
#             batchLoss.append(loss.item())
#             guesses = torch.argmax(output,1)
#             if pvalidateOut : print('output: \n',output)
#             corrects = yval.clone().detach() == guesses
#             if pvalidateOut: print(yval.clone().detach(), guesses)
#             if pvalidateOut: print(corrects.detach())
#             if pvalidateOut: print('===========================\n\n\n')
#             unbalanced.append([1 if correct else 0 for correct in corrects.detach()])
        
#             for i, ans in enumerate(yval):
#                 if pbalApp: print(i,ans, guesses[i], corrects[i])
#                 balanced[ans].append(corrects[i])
        
#         balanced = [np.mean(i) for i in balanced]
#         balancedAccuracy = np.mean(balanced)
        
#         print(f'The total balanced accuracy for validation was {balancedAccuracy}')
#         print(f'The validation loss was :   {epoch+1}/{epochs} was {np.mean(batchLoss)}')
#         print(f'The unbalanced validation accuracy is {np.mean(unbalanced)}')
#         print(f'The accuracy for each is {balanced}')       
        
        
#         if checkClone:
#             fives = modelModule(*modelArgs, **modelKwargs)
#             fives.load_state_dict(torch.load(PATH))
#             s=2
#             feature_extraction1 = [child for child in model.children()]
#             print(feature_extraction1[s])
#             feature_extraction2 = [child for child in fives.children()]
#             print(feature_extraction2[s])
#             print(torch.max(feature_extraction1[s].weight - feature_extraction2[s].weight).detach())
#             print(torch.min(feature_extraction1[s].weight - feature_extraction2[s].weight).detach())
#         print('\n\n=============End Epoch==============\n\n')
#     if pstateDict: print(opt.state_dict())


#     return model


In [48]:
# [weights1[i] + weights2[i] for i in range(5)]
trainArgs = [logRegWithHidden, train1, labels1, None, train1, labels1, None]
modelArgs = [4096*4, 2048*4]
lr = 0.01
# pvalidateOut=True, pbalApp=True, 
%time newModel = trainer(*trainArgs, *modelArgs, lr = lr, drop1=0)
lr

The training loss for epoch 1/40 was 1.6036966030414288
The total balanced accuracy for validation was 0.2
The validation loss was :   1/40 was 1.5972699827449337
The unbalanced validation accuracy is 0.2
The accuracy for each is [1.0, 0.0, 0.0, 0.0, 0.0]


=============End Epoch==============


The training loss for epoch 2/40 was 1.592959110553448
The total balanced accuracy for validation was 0.2535031847133758
The validation loss was :   2/40 was 1.5876273387556623
The unbalanced validation accuracy is 0.2535031847133758
The accuracy for each is [1.0, 0.0, 0.0, 0.0, 0.267515923566879]


=============End Epoch==============


The training loss for epoch 3/40 was 1.5836453162706816
The total balanced accuracy for validation was 0.30573248407643316
The validation loss was :   3/40 was 1.578737127249408
The unbalanced validation accuracy is 0.3057324840764331
The accuracy for each is [1.0, 0.0, 0.0, 0.0, 0.5286624203821656]


=============End Epoch==============


The training loss for

The total balanced accuracy for validation was 0.49044585987261147
The validation loss was :   24/40 was 1.4317874367070047
The unbalanced validation accuracy is 0.49044585987261147
The accuracy for each is [0.9872611464968153, 0.49044585987261147, 0.0, 0.19745222929936307, 0.7770700636942676]


=============End Epoch==============


The training loss for epoch 25/40 was 1.4284414052963257
The total balanced accuracy for validation was 0.5006369426751592
The validation loss was :   25/40 was 1.4283350535259125
The unbalanced validation accuracy is 0.5006369426751592
The accuracy for each is [0.9872611464968153, 0.49044585987261147, 0.0, 0.25477707006369427, 0.7707006369426752]


=============End Epoch==============


The training loss for epoch 26/40 was 1.4250577321419349
The total balanced accuracy for validation was 0.513375796178344
The validation loss was :   26/40 was 1.4249830783552424
The unbalanced validation accuracy is 0.513375796178344
The accuracy for each is [0.9808917197

0.01

In [49]:
from datetime import datetime
acc = 
PATH = f'savedModels/lr{lr}acc{acc}time{datetime.now().strftime("%d_%H:%M")}.pth'
torch.save(newModel.state_dict(), PATH)
print('REMEMBER TO DELETE YOUR ACCURACY SO THE NEXT PERSON REMEMBERS TO WRITE THEIRS')

REMEMBER TO DELETE YOUR ACCURACY SO THE NEXT PERSON REMEMBERS TO WRITE THEIRS
